# <font color='black'> Homework group: </font>
Nicholas Butt (20757197) and Abby Tien (20758949)

# <font color='orange'> Imports </font>


In [1]:
import pandas as pd
import numpy as np
from numpy.random import default_rng
from plotnine import *

rng = default_rng(13)
import warnings
warnings.filterwarnings('ignore')

In [2]:
# ==============================================================================
# ========== You do not need to make changes to this cell ======================
# ==============================================================================
def runModel(p,m,c):

    theProbOfSnowyDay = p
    theMeanAmountOfSnow = m
    theStdevAmountOfSnow = 0.5428 
    theCostOfInsuficientPlowing = c
    theEstimateLoss = 0
    plowing_capacity = 2
    

    # Get hold of the probability that it will snow today for this scenario
    theProbabilityForThisScenario = theProbOfSnowyDay 


    # This is convoluted but quicker way to calculate losses due to lack of snow plowing capacity
    v = (rng.random(32) < theProbabilityForThisScenario) * rng.normal(theMeanAmountOfSnow,theStdevAmountOfSnow,32)


    theGap = [] # difference between accumulated snow and plowing capacity
    for t in range(32):
        if t == 0:
            theGap.append(max(v[t] - plowing_capacity,0))
        else:
            theGap.append(max(theGap[t-1] + v[t] - plowing_capacity,0))
    
    return np.sum((np.array(theGap) > 0) * theCostOfInsuficientPlowing)

In [3]:
# Consider that the probability of snowfall (P) can be between 0.25-0.75 (uniformly distributed)
# Consider that the mean amount of snow (M) can be between 2-5 cm (uniformly distributed)
# Consider that the cost per day of insufficient plowing (C) can be between $25000-75000 (uniformly distributed)

num_rows = 10000

# Create matrix A like this
# P    |  M  |  C
#_____________________
# 0.5  |  3  |  40000
# 0.4  |  4  |  65000
#.....
# N rows
A = [list(rng.uniform(low=0.25, high=0.75, size=num_rows)),
     list(rng.uniform(low=2, high=5, size=num_rows)),
     list(rng.uniform(low=25000, high=75000, size=num_rows))]


# Create matrix B like this
# P    |  M  |  C
#_____________________
# 0.6  |  5  |  50000
# 0.3  |  2  |  35000
#.....
# N rows
B = [list(rng.uniform(low=0.25, high=0.75, size=num_rows)),
     list(rng.uniform(low=2, high=5, size=num_rows)),
     list(rng.uniform(low=25000, high=75000, size=num_rows))]

'''
# Hint: A and B must contain different values
'''


# Calculate outputs calculated on A
f_a = []
for i in range(len(A[0])):
  f_a.append(runModel(A[0][i],A[1][i],A[2][i]))
# Calculate outputs calculated on B
f_b = []
for i in range(len(B[0])):
  f_b.append(runModel(B[0][i],B[1][i],B[2][i]))


# Matrix AB_1
AB_1 = np.copy(A) # use copy or you will overwrite A
AB_1[0] = B[0]
# Matrix AB_2
AB_2 = np.copy(A) # use copy or you will overwrite A
AB_2[1] = B[1]
# Matrix AB_3
AB_3 = np.copy(A) # use copy or you will overwrite A
AB_3[2] = B[2]


# Outputs calculated on AB_1
f_1 = []
for i in range(len(AB_1[0])):
  f_1.append(runModel(AB_1[0][i],AB_1[1][i],AB_1[2][i]))
# Outputs calculated on AB_2
f_2 = []
for i in range(len(AB_2[0])):
  f_2.append(runModel(AB_2[0][i], AB_2[1][i], AB_2[2][i]))
# Outputs calculated on AB_3
f_3 = []
for i in range(len(AB_3[0])):
  f_3.append(runModel(AB_3[0][i], AB_3[1][i], AB_3[2][i]))

In [4]:
# Calculate the total variance
Var = 0
for i in range(num_rows):
  Var += (f_b[i] - f_a[i])**2

Var /= 2*num_rows
print(f'The total variance is: {Var}')

The total variance is: 285370823552.2118


In [5]:
# Calculate the first order Sobol Indexes

# Probability of snowfall first order Sobal index
Var1 = 0
for i in range(num_rows):
  Var1 += f_b[i] * (f_1[i] - f_a[i])
Var1 /= num_rows
fo_sobal_p = Var1/Var

# mean amount of snowfall first order Sobal index
Var2 = 0
for i in range(num_rows):
  Var2 += f_b[i] * (f_2[i] - f_a[i])
Var2 /= num_rows
fo_sobal_m = Var2/Var

# cost per day of insufficient snow clearing capacity first order Sobal index
Var3 = 0
for i in range(num_rows):
  Var3 += f_b[i] * (f_3[i] - f_a[i])
Var3 /= num_rows
fo_sobal_c = Var3/Var

print(f'The first order Sobal index for m is: {fo_sobal_p}')
print(f'The first order Sobal index for p is: {fo_sobal_m}')
print(f'The first order Sobal index for c is: {fo_sobal_c}')

The first order Sobal index for m is: 0.2766234700442582
The first order Sobal index for p is: 0.24672974203257605
The first order Sobal index for c is: 0.30925787885006445


In [6]:
# Calculate the total Sobol Indexes

# Probability of snowfall total Sobal index
E1 = 0
for i in range(num_rows):
  E1 += (f_a[i] - f_1[i])**2
E1 /= (2*num_rows)
total_sobal_p = E1/Var

# mean amount of snowfall total Sobal index
E2 = 0
for i in range(num_rows):
  E2 += (f_a[i] - f_2[i])**2
E2 /= (2*num_rows)
total_sobal_m = E2/Var

# cost per day of insufficient snow clearing capacity total Sobal index
E3 = 0
for i in range(num_rows):
  E3 += (f_a[i] - f_3[i])**2
E3 /= (2*num_rows)
total_sobal_c = E3/Var

print(f'The total Sobal index for m is: {total_sobal_p}')
print(f'The total Sobal index for p is: {total_sobal_m}')
print(f'The total Sobal index for c is: {total_sobal_c}')

The total Sobal index for m is: 0.4287127114930639
The total Sobal index for p is: 0.3788148306084228
The total Sobal index for c is: 0.520652409134741


In [7]:
# Identify which parameters contribute the most to the variance of the model

In [8]:
'''
In my solution, the total variance calculated on 10,000 points was: 281851096953.87476
The first order Sobol indices were: 0.2313, 0.2112, and 0.3025
The total Sobol indices were: 0.4309, 0.3934, 0.5145

All of these values may be slightly different for you because they are approximations.
'''

'\nIn my solution, the total variance calculated on 10,000 points was: 281851096953.87476\nThe first order Sobol indices were: 0.2313, 0.2112, and 0.3025\nThe total Sobol indices were: 0.4309, 0.3934, 0.5145\n\nAll of these values may be slightly different for you because they are approximations.\n'